In [77]:
import os
from dotenv import load_dotenv, find_dotenv
import unidecode
from lyricsgenius import Genius
import spotipy
import spotipy.util as util
import text2emotion as te
import pandas as pd
from difflib import SequenceMatcher
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/yann/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [78]:
env_path = find_dotenv()
load_dotenv(env_path)

True

# Get the data from spotify

In [79]:
data = pd.read_csv("../raw_data/data.csv")

"'Dennis Day'"

# Testing Genius API

In [82]:
# loading needed credentials
genius_token = os.environ.get('LYRICSGENIUS_ACCESS_TOKEN')
# instantiate the class with some parameters
genius = Genius(genius_token, 
                timeout=120, 
                remove_section_headers=True, 
                sleep_time=0.3, 
                skip_non_songs=True)

In [101]:
data

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.98200,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.6650,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.73200,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.1600,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.96100,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.1010,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.96700,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.3810,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.95700,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.2290,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170648,0.6080,2020,0.08460,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...",0.786,301714,0.808,0,0KkIkfsLEJbrcIhYsCL7L5,0.000289,7,0.0822,-3.702,1,China,72,2020-05-29,0.0881,105.029
170649,0.7340,2020,0.20600,['Ashnikko'],0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.1010,-6.020,1,Halloweenie III: Seven Days,68,2020-10-23,0.0605,137.936
170650,0.6370,2020,0.10100,['MAMAMOO'],0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.2580,-2.226,0,AYA,76,2020-11-03,0.0809,91.688
170651,0.1950,2020,0.00998,['Eminem'],0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.6430,-7.161,1,Darkness,70,2020-01-17,0.3080,75.055


In [107]:
data.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [163]:
for d in range(2):
    data_song = data.name[d]
    data_artist = data.artists[d].replace("[","").replace("]","").replace(",","")

    lyrics = check_song(data_artist,data_song)
    if lyrics != None:
        emotions  = jars_get_emotions(lyrics)
        sentiments  = jars_get_sentiments(lyrics)
    else:
        emotions = 0
    # print(emotions)
    # print(sentiments)

Searching for "Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve" by 'Sergei Rachmaninoff' 'James Levine' 'Berliner Philharmoniker'...
No results found for: 'Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve 'Sergei Rachmaninoff' 'James Levine' 'Berliner Philharmoniker''
None
Searching for "Clancy Lowered the Boom" by 'Dennis Day'...
Done.


NameError: name 'user_input_song' is not defined

In [161]:
def check_song(data_artist,data_song):
    # search with the user input (artist + song title)
    song = genius.search_song(title = data_song, artist = data_artist)
    if song == None:
        lyrics = None
        print(lyrics)
    else:
        full_title = song.to_dict()['full_title']
        test_api_response = unidecode.unidecode(full_title.replace('\xa0', ' ').replace('\u200b', ' ').lower())
        test_api_response = test_api_response.split(' by ')
        song_similar = similar(test_api_response[0], unidecode.unidecode(data_song.lower()))
        artist_similar = similar(test_api_response[1], unidecode.unidecode(data_artist.lower()))
        # song.to_dict()['id']
        # characters_to_remove = len(song.to_dict()['title'] + ' Lyrics')
        if song_similar > 0.9:
            lyrics = genius.lyrics(song.to_dict()['id'])
        else:
            lyrics = None
    
    return lyrics

def jars_get_emotions(lyrics):
    emotions = te.get_emotion(lyrics)
    return emotions

def jars_get_sentiments(lyrics):
    sid = SentimentIntensityAnalyzer()
    sentiments = sid.polarity_scores(lyrics)    
    return sentiments    
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
emo

{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.43, 'Sad': 0.24, 'Fear': 0.19}

In [ ]:
# getting lyrics
lyrics = genius.lyrics(song_id)[characters_to_remove:-5]
lyrics

NameError: name 'song_id' is not defined

In [ ]:
# simple creanup for text2emotion testing
cleaned_lyrics = lyrics.replace('\n', ' ')
cleaned_lyrics

"  So close, no matter how far Couldn't be much more from the heart Forever trusting who we are  And nothing else matters  Never opened myself this way Life is ours, we live it our way All these words I don't just say And nothing else matters  Trust I seek and I find in you Everyday, for us, something new Open mind for a different view  And nothing else matters  Never cared for what they do Never cared for what they know But I know  So close, no matter how far It couldn't be much more from the heart Forever trusting who we are  And nothing else matters  Never cared for what they do Never cared for what they know But I know You might also like  I never opened myself this way Life is ours, we live it our way All these words I don't just say  And nothing else matters  Trust I seek and I find in you Everyday, for us, something new Open mind for a different view  And nothing else matters  Never cared for what they say Never cared for games they play  Never cared for what they do Never cared

# Testing Spotify API

In [ ]:
# needed credentials and permissions (scope)
cid = os.environ.get('SPOTIFY_CLIENT_ID')
secret = os.environ.get('SPOTIFY_CLIENT_SECRET')
redirect_uri = os.environ.get('SPOTIPY_REDIRECT_URI')
scope = 'playlist-modify-private'
username = None

In [ ]:
# testing locally for user authentication, later development is needed
spotify_token = util.prompt_for_user_token(username,
                                           scope,
                                           cid,
                                           secret,
                                           redirect_uri)

sp = spotipy.Spotify(auth=spotify_token)

In [ ]:
# getting user id
user_id = sp.me()['id']

In [ ]:
# creating a private playlist
playlist = sp.user_playlist_create(user_id, 
                                   'test', 
                                   public=False, 
                                   collaborative=False, 
                                   description='Testing')

In [ ]:
# obtaining playlist id
playlist_id = playlist['id']
playlist_id

'1Q71YBTedi64Hy1gnUe5Tg'

In [ ]:
# artist URI hardcoded for testing purposes
# this top_tracks_list will be replaced by the IDs 
# extracted by the recommender
top_tracks = sp.artist_top_tracks('spotify:artist:2NFrAuh8RQdQoS7iYFbckw')

In [ ]:
# get a list of tracks URI
top_tracks_list = [tracks['uri'] for tracks in top_tracks['tracks']]
top_tracks_list

['spotify:track:4loXMor75kKVBB03ygwDlh',
 'spotify:track:4PMmwowVLOajPdiKnrU1vK',
 'spotify:track:3nNNYRBtkNsKjertmtTRMO',
 'spotify:track:6tJPdGKrbAeyhkkhn44RHR',
 'spotify:track:6S10TQTWVy1Zf3bscMFBlU',
 'spotify:track:1K0urJwRWgKsdufGEdkXP7',
 'spotify:track:24B4TFlEOJzzqb33L4sbI5',
 'spotify:track:4um6CPDIxnNWSEbj3LJQhQ',
 'spotify:track:5s9eoyEyoeEj3fqtt0pQoQ',
 'spotify:track:2hj7EbKUGH0xtAzv2q22dZ']

In [ ]:
# add songs to playlist
sp.user_playlist_add_tracks(username, 
                            playlist_id, 
                            top_tracks_list, 
                            position=None)

{'snapshot_id': 'MixhNDEzMzEzMTdmNTExMjNlYzlmMTMwMDg5M2NkNDY3OGU1YzAwNWIz'}